# 📘 Final Project - Recommender Systems

### 📌 Submitted by:
#### 👤 1. Ebenezer Daniel  
#### 👤 2. Raja Prabhakaran  
#### 👤 3. Nitheesh Samiappan  

## 📚 **Dataset Details: Goodreads Book Datasets (10M)**
### 📌 Source:
🔗 **[Goodreads Book Datasets (10M) on Kaggle](https://www.kaggle.com/datasets/bahramjannesarr/goodreads-book-datasets-10m/data)**

### 📊 **Overview**
This dataset contains **book information and user ratings** from **Goodreads**, a popular online book review platform. It includes metadata on **millions of books**, user-generated reviews, and rating distributions.

### 📂 **Files Included**
- **Books Data**: Contains metadata such as title, author, publisher, publication year, and ratings.
- **User Ratings Data**: Includes detailed breakdowns of how users rated books (1-star to 5-star distributions).

### 🔍 **Dataset Attributes**
| Column Name         | Description |
|---------------------|------------|
| **Id**             | Unique book identifier |
| **Name**           | Title of the book |
| **Authors**        | Author(s) of the book |
| **ISBN**           | International Standard Book Number (if available) |
| **Publisher**      | Name of the publishing house |
| **PagesNumber**    | Number of pages in the book |
| **PublishYear**    | Year of publication |
| **PublishMonth**   | Month of publication |
| **PublishDay**     | Day of publication |
| **Language**       | Language of the book |
| **CountsOfReview** | Total number of user reviews |
| **Rating**         | Average user rating (1-5) |
| **RatingDist1**    | Number of 1-star ratings |
| **RatingDist2**    | Number of 2-star ratings |
| **RatingDist3**    | Number of 3-star ratings |
| **RatingDist4**    | Number of 4-star ratings |
| **RatingDist5**    | Number of 5-star ratings |
| **RatingDistTotal** | Total number of ratings |

### 📈 **Size of the Dataset**
- **Number of books:** ~10 million  
- **Number of user ratings:** ~25 million  

### 🛠 **Preprocessing Done**
- **Handling Missing Values:** Some books are missing ISBNs, publishers, or languages.
- **Standardizing Column Names:** Ensured consistency across multiple files.
- **Data Cleaning:** Removed duplicates, merged similar columns (e.g., `PagesNumber` and `pagesNumber`).

---

### 📌 **Why This Dataset?**
- Large-scale book rating data enables **recommendation system development**.
- Rich metadata for **book analysis, trend discovery, and user preferences**.
- Real-world **collaborative filtering & machine learning applications**.

---

📢 *This dataset serves as the foundation for our book recommender system and data analysis!* 🚀


In [3]:
import importlib
import subprocess
import sys

# List of required libraries
required_libraries = [
    "pandas", "numpy", "matplotlib", "seaborn", "scipy", 
    "nltk", "scikit-learn", "scikit-surprise"  # Use scikit-surprise instead of surprise
]

# Function to check and install missing libraries
def check_and_install(libraries):
    for lib in libraries:
        try:
            importlib.import_module(lib)
        except ImportError:
            print(f"Installing {lib}...")
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
            except subprocess.CalledProcessError:
                print(f"Failed to install {lib}. Trying Conda (if applicable)...")
                try:
                    subprocess.check_call(["conda", "install", "-c", "conda-forge", lib, "-y"])
                except subprocess.CalledProcessError:
                    print(f"Could not install {lib}. Please install it manually.")

# Check and install missing libraries
check_and_install(required_libraries)

print("All required libraries are installed.")


Installing scikit-learn...
Installing scikit-surprise...
All required libraries are installed.


In [310]:
# Jupyter magic command for inline plots
%matplotlib inline

# Import libraries
import pandas as pd
import glob
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings
warnings.simplefilter('ignore')

print("All required libraries are successfully imported.")

All required libraries are successfully imported.


In [5]:
# Define the folder where your CSV files are stored
csv_folder = "Datasets/" 

# Use glob to get only CSV files that start with 'book'
csv_book_files = glob.glob(os.path.join(csv_folder, "book*.csv"))
csv_user_files = glob.glob(os.path.join(csv_folder, "user*.csv"))

# Load and concatenate all "book" CSV files into a single DataFrame
# Function to read CSV files and standardize column names before combining
def load_and_standardize_csv(file_list):
    df_list = []
    for file in file_list:
        df = pd.read_csv(file)
        df.columns = df.columns.str.lower()
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

# Load and combine "book" and "user" CSV files
df_books = load_and_standardize_csv(csv_book_files)
df_users = load_and_standardize_csv(csv_user_files)

# Rename specific mismatched columns for consistency
df_books.rename(columns={'pagesnumber': 'pagesNumber'}, inplace=True)

print(f"Combined 'book' DataFrame shape: {df_books.shape}")
print(f"Combined 'user' DataFrame shape: {df_users.shape}")

Combined 'book' DataFrame shape: (1850310, 20)
Combined 'user' DataFrame shape: (362596, 3)


In [6]:
#store the merged data into a csv files for both books and user
df_books.to_csv('combined_books_data.csv', index=False) # Causes memory error
df_users.to_csv('combined_users_data.csv', index=False)

In [79]:
#copy the data frame and keep the original data for further use
df_books_original = df_books.copy()
df_users_original = df_users.copy()
print('Original data copied for future use. DONE!!!')

Original data copied for future use. DONE!!!


### Preprocessing & Data Cleaning

In [81]:
# find the unique ID and ISBN
unique_ids = df_books['id'].nunique()
unique_isbns = df_books['isbn'].nunique()
print(f"Unique IDs: {unique_ids}")
print(f"Unique ISBNs: {unique_isbns}")

Unique IDs: 1850115
Unique ISBNs: 1844192


In [82]:
# find duplicates across all rows
duplicate_rows = df_books.duplicated().sum()
print(f"Duplicate Rows: {duplicate_rows}")

Duplicate Rows: 112


In [83]:
# find missing ISBNs & ID
missing_isbns = df_books['isbn'].isna().sum()
print(f"Missing ISBNs: {missing_isbns}")

missing_id = df_books['id'].isna().sum()
print(f"Missing Id: {missing_id}")

Missing ISBNs: 5923
Missing Id: 0


In [85]:
# Drop duplicate rows
df_books_cleaned = df_books.drop_duplicates()
print(f"After removing fully duplicated rows: {df_books_cleaned.shape[0]} records remaining.")

# Keep only the first occurrence of each ID
df_books_cleaned = df_books_cleaned.drop_duplicates(subset=['id'], keep='first')
print(f"After ensuring unique IDs: {df_books_cleaned.shape[0]} records remaining.")

After removing fully duplicated rows: 1850198 records remaining.
After ensuring unique IDs: 1850115 records remaining.


In [87]:
# Check for missing (NaN) values across all columns
missing_values = df_books_cleaned.isna().sum()
missing_values

id                             0
name                           0
ratingdist1                    0
pagesNumber                    0
ratingdist4                    0
ratingdisttotal                0
publishmonth                   0
publishday                     0
publisher                  17821
countsofreview                 0
publishyear                    0
language                 1598369
authors                        0
rating                         0
ratingdist2                    0
ratingdist5                    0
isbn                        5922
ratingdist3                    0
description               678927
count of text reviews    1440418
dtype: int64

In [94]:
# Dropping all the unnesscary columns
df_books_cleaned.drop(columns=['count of text reviews', 'isbn'], inplace=True)
df_books_cleaned.head(5)

,id,name,ratingdist1,pagesNumber,ratingdist4,ratingdisttotal,publishmonth,publishday,publisher,countsofreview,publishyear,language,authors,rating,ratingdist2,ratingdist5,ratingdist3,description
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,2006,eng,J.K. Rowling,4.57,2:25317,5:1546466,3:159960,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,2004,eng,J.K. Rowling,4.50,2:37005,5:1493113,3:211781,NaN
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,1:108202,309,4:1513191,total:6587388,1,11,Scholastic Inc,75911,2003,eng,J.K. Rowling,4.47,2:130310,5:4268227,3:567458,NaN
3,4,Harry Potter and the Chamber of Secrets (Harry...,1:11896,352,4:706082,total:2560657,1,11,Scholastic,244,2003,eng,J.K. Rowling,4.42,2:49353,5:1504505,3:288821,NaN
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,1:10128,435,4:630534,total:2610317,1,5,Scholastic Inc.,37093,2004,eng,J.K. Rowling,4.57,2:24849,5:1749958,3:194848,NaN


In [102]:
df_books_cleaned['publisher'].fillna("Unknown", inplace=True)
df_books_cleaned['language'].fillna(df_books_cleaned['language'].mode()[0], inplace=True)

In [104]:
df_books_cleaned.head(2)

,id,name,ratingdist1,pagesNumber,ratingdist4,ratingdisttotal,publishmonth,publishday,publisher,countsofreview,publishyear,language,authors,rating,ratingdist2,ratingdist5,ratingdist3,description
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,2006,eng,J.K. Rowling,4.57,2:25317,5:1546466,3:159960,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,2004,eng,J.K. Rowling,4.50,2:37005,5:1493113,3:211781,NaN


In [106]:
# takes 10 to 30 mins to Run !!!!
from summa import summarizer

# Step 1: Create a mapping of authors to non-null descriptions
author_descriptions = df_books_cleaned.groupby('authors')['description'].apply(lambda x: " ".join(x.dropna().values[:5]))

# Step 2: Fill missing descriptions in bulk
def generate_summary(author, desc):
    if pd.isna(desc) and author in author_descriptions:
        return summarizer.summarize(author_descriptions[author], words=50)
    return desc

# Apply faster in bulk (vectorized)
df_books_cleaned['description'] = df_books_cleaned.apply(lambda row: generate_summary(row['authors'], row['description']), axis=1)

In [220]:
# find different types of rating in the rating colum
ratings = df_users['rating'].unique()
print(ratings)

['it was amazing' 'really liked it' 'liked it' 'did not like it'
 'it was ok' "This user doesn't have any rating"]


In [222]:
rating_mapping = {
    "it was amazing": 5,
    "really liked it": 4,
    "liked it": 3,
    "it was ok": 2,
    "did not like it": 1,
    "This user doesn't have any rating": 0  # Convert to NaN or remove
}

# create a numeric rating colum
df_users['rating in numbers'] = df_users['rating'].map(rating_mapping)
df_users.head(5)

,id,name,rating,rating in numbers
0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing,5
1,1,The Restaurant at the End of the Universe (Hit...,it was amazing,5
2,1,Siddhartha,it was amazing,5
3,1,The Clock of the Long Now: Time and Responsibi...,really liked it,4
4,1,"Ready Player One (Ready Player One, #1)",really liked it,4


In [224]:
# copying the data frame
df_users_copy = df_users.copy()
df_books_copy = df_books_cleaned.copy() 

In [225]:
# renaming the columns in both the data frame
df_books_copy.rename(columns={'id': 'book_id'}, inplace=True)
df_users_copy.rename(columns={'id': 'user_id'}, inplace=True)

In [226]:
df_users_copy.head()

,user_id,name,rating,rating in numbers
0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing,5
1,1,The Restaurant at the End of the Universe (Hit...,it was amazing,5
2,1,Siddhartha,it was amazing,5
3,1,The Clock of the Long Now: Time and Responsibi...,really liked it,4
4,1,"Ready Player One (Ready Player One, #1)",really liked it,4


In [227]:
df_books_copy.head(5)

,book_id,name,ratingdist1,pagesNumber,ratingdist4,ratingdisttotal,publishmonth,publishday,publisher,countsofreview,publishyear,language,authors,rating,ratingdist2,ratingdist5,ratingdist3,description
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,2006,eng,J.K. Rowling,4.57,2:25317,5:1546466,3:159960,Harry kan dan ook niet wachten tot hij terug m...
1,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,2004,eng,J.K. Rowling,4.50,2:37005,5:1493113,3:211781,Harry kan dan ook niet wachten tot hij terug m...
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,1:108202,309,4:1513191,total:6587388,1,11,Scholastic Inc,75911,2003,eng,J.K. Rowling,4.47,2:130310,5:4268227,3:567458,Harry kan dan ook niet wachten tot hij terug m...
3,4,Harry Potter and the Chamber of Secrets (Harry...,1:11896,352,4:706082,total:2560657,1,11,Scholastic,244,2003,eng,J.K. Rowling,4.42,2:49353,5:1504505,3:288821,Harry kan dan ook niet wachten tot hij terug m...
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,1:10128,435,4:630534,total:2610317,1,5,Scholastic Inc.,37093,2004,eng,J.K. Rowling,4.57,2:24849,5:1749958,3:194848,Harry kan dan ook niet wachten tot hij terug m...


In [228]:
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

df_users_copy['clean_name'] = df_users_copy['name'].apply(clean_text)
df_books_copy['clean_name'] = df_books_copy['name'].apply(clean_text)

In [229]:
# mapping name and bookid
name_to_book_id_map = df_books_copy.set_index('clean_name')['book_id'].to_dict()
df_users_copy['book_id'] = df_users_copy['clean_name'].map(name_to_book_id_map)

In [230]:
# finding the matched count in both the books and users dataframe
matched_count = df_users_copy['book_id'].notna().sum()
print(f"Directly matched count: {matched_count}")

Directly matched count: 234734


In [231]:
# finding the unmatched count in both the books and users dataframe
unmatched_count = df_users_copy['book_id'].isna().sum()
print(f"Directly not matched count: {unmatched_count}")

Directly not matched count: 127862


In [235]:
# First, map directly where names match exactly (FAST)
df_users_copy['book_id'] = df_users_copy['clean_name'].map(name_to_book_id_map)

# Count directly matched entries
matched_count = df_users_copy['book_id'].notna().sum()
print(f"Directly matched count: {matched_count}")


Directly matched count: 234734


In [237]:
df_users_copy.head(5)

,user_id,name,rating,rating in numbers,clean_name,book_id
0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing,5,agile web development with rails a pragmatic g...,45.0
1,1,The Restaurant at the End of the Universe (Hit...,it was amazing,5,the restaurant at the end of the universe hitc...,862825.0
2,1,Siddhartha,it was amazing,5,siddhartha,828548.0
3,1,The Clock of the Long Now: Time and Responsibi...,really liked it,4,the clock of the long now time and responsibility,1788479.0
4,1,"Ready Player One (Ready Player One, #1)",really liked it,4,ready player one ready player one 1,NaN


In [238]:
# storing it in a csv file
unmatched_rows = df_users_copy[df_users_copy['book_id'].isna()]
unmatched_rows.to_csv("unmatched_books.csv", index=False)

In [239]:
# storing it in a csv file
matched_rows = df_users_copy[df_users_copy['book_id'].notna()]
matched_rows.to_csv("matched_books.csv", index=False)

In [248]:
unique_book_ids = df_users_copy['book_id'].nunique()
print(f"Total unique book IDs: {unique_book_ids}")

Total unique book IDs: 51336


In [250]:
missing_values = matched_rows.isna().sum()
missing_values

user_id              0
name                 0
rating               0
rating in numbers    0
clean_name           0
book_id              0
dtype: int64

In [252]:
# Check for missing (NaN) values across all columns
missing_values = df_books_copy.isna().sum()
missing_values

book_id            0
name               0
ratingdist1        0
pagesNumber        0
ratingdist4        0
ratingdisttotal    0
publishmonth       0
publishday         0
publisher          0
countsofreview     0
publishyear        0
language           0
authors            0
rating             0
ratingdist2        0
ratingdist5        0
ratingdist3        0
description        0
clean_name         0
dtype: int64

### Simple Recommender Systems

#### For unique book ids

In [293]:
# Step 1: Extract unique book IDs from users_df
unique_book_ids = matched_rows['book_id'].unique()

# Step 2: Filter book_df to keep only books that exist in users_df
filtered_books_df = df_books_copy[df_books_copy['book_id'].isin(unique_book_ids)].copy()

# Step 3: Ensure only unique book entries remain
filtered_books_df = filtered_books_df.drop_duplicates(subset=['book_id'])

In [337]:
def compute_weighted_recommendations(books_df, quantile_threshold=0.90):
    # Compute the mean rating across all books
    mean_rating = books_df['rating'].mean()

    # Determine the minimum number of reviews required for a book to be considered
    min_no_of_reviews = books_df['countsofreview'].quantile(quantile_threshold)

    # Filter books with at least `min_no_of_reviews` reviews
    popular_books = books_df[books_df['countsofreview'] >= min_no_of_reviews].copy()

    # Define the IMDB Weighted Rating function
    def weighted_rating(x, m=min_no_of_reviews, C=mean_rating):
        v = x['countsofreview']  # Number of reviews
        R = x['rating']          # Average rating
        return (v / (v + m) * R) + (m / (v + m) * C)

    # Apply the weighted rating formula
    popular_books['score'] = popular_books.apply(weighted_rating, axis=1)

    # Sort books based on score
    recommended_books = popular_books.sort_values('score', ascending=False)

    return recommended_books

In [343]:
recommended_books = compute_weighted_recommendations(filtered_books_df)

In [341]:
def display_recommended_books(recommended_books, top_n=10):
    # Extract the top N recommended books
    top_books = recommended_books[['name', 'authors', 'rating', 'countsofreview', 'score']].head(top_n)

    # Create a lively, well-formatted table with emojis and colors
    table_str = tabulate(
        top_books,
        headers=["📖 Book Title", "✍️ Author(s)", "⭐ Rating", "🗳️ Review Count", "🏆 Score"],
        tablefmt="fancy_grid",
        showindex=False
    )

    # Print the recommended books in a lively table format
    print("\n🎉📚 **Top {} Recommended Books** 📚🎉\n".format(top_n))
    print(table_str)

In [349]:
display_recommended_books(recommended_books, top_n=10)  # Show Top 10 Books


🎉📚 **Top 10 Recommended Books** 📚🎉

╒═════════════════════════════════════════════════════════════════════════════╤════════════════════╤═════════════╤══════════════════╤════════════╕
│ 📖 Book Title                                                               │ ✍️ Author(s)        │   ⭐ Rating │   🗳️ Review Count │   🏆 Score │
╞═════════════════════════════════════════════════════════════════════════════╪════════════════════╪═════════════╪══════════════════╪════════════╡
│ Harry Potter Series Box Set (Harry Potter, #1-7)                            │ J.K. Rowling       │        4.74 │             6522 │    4.71786 │
├─────────────────────────────────────────────────────────────────────────────┼────────────────────┼─────────────┼──────────────────┼────────────┤
│ The Complete Calvin and Hobbes                                              │ Bill Watterson     │        4.82 │              940 │    4.67414 │
├─────────────────────────────────────────────────────────────────────────────┼───

#### For the entire books data frame

In [353]:
recommended_books = compute_weighted_recommendations(df_books_copy)

In [355]:
display_recommended_books(recommended_books, top_n=10)  # Show Top 10 Books


🎉📚 **Top 10 Recommended Books** 📚🎉

╒═══════════════════════════════════════════════════════════════════╤═══════════════════╤═════════════╤══════════════════╤════════════╕
│ 📖 Book Title                                                     │ ✍️ Author(s)       │   ⭐ Rating │   🗳️ Review Count │   🏆 Score │
╞═══════════════════════════════════════════════════════════════════╪═══════════════════╪═════════════╪══════════════════╪════════════╡
│ The Complete Calvin and Hobbes                                    │ Bill Watterson    │        4.82 │              940 │    4.80375 │
├───────────────────────────────────────────────────────────────────┼───────────────────┼─────────────┼──────────────────┼────────────┤
│ Mark of the Lion Trilogy                                          │ Francine Rivers   │        4.77 │              757 │    4.75038 │
├───────────────────────────────────────────────────────────────────┼───────────────────┼─────────────┼──────────────────┼────────────┤
│ Harry Pott